In [1]:
# 6_polypharmacy_pbpk_inputs.ipynb
# Notebook to prepare PBPK-ready input tables for polypharmacy scenarios from harmonized pediatric PK dataset

# ----------------------------------
# 1. Import libraries
# ----------------------------------
import pandas as pd
import os

# ----------------------------------
# 2. Load harmonized dataset
# ----------------------------------
HARMONIZED_FILE = '/data/curated/pediatric_pk_master_harmonized.csv'
df = pd.read_csv(HARMONIZED_FILE)
print('Harmonized dataset preview:')
display(df.head())

# ----------------------------------
# 3. Assign pediatric age groups (if not already present)
# ----------------------------------
AGE_GROUPS = {
    'preterm_neonate': (0, 0.08),
    'term_neonate': (0.08, 0.28),
    'infant': (0.28, 1),
    'child': (1, 12),
    'adolescent': (12, 18),
    'adult': (18, 100)
}

def assign_age_group(age):
    for group, (start, end) in AGE_GROUPS.items():
        if pd.notna(age) and start <= age < end:
            return group
    return 'unknown'

df['age_group_pbpk'] = df['age_years'].apply(assign_age_group)

# ----------------------------------
# 4. Identify polypharmacy scenarios
# ----------------------------------
# Assuming 'subject_id' can have multiple drugs
poly_df = df.groupby(['subject_id', 'age_group_pbpk']).agg({
    'drug': lambda x: list(x.unique()),
    'cl_l_per_h_per_kg': 'mean',  # average clearance across drugs
    'weight_kg': 'mean',          # average weight
}).reset_index()

# Add a flag for subjects with multiple drugs
poly_df['polypharmacy'] = poly_df['drug'].apply(lambda x: len(x) > 1)

# ----------------------------------
# 5. Save PBPK-ready polypharmacy inputs
# ----------------------------------
PBPK_POLY_DIR = '/data/curated/pbpk_inputs/polypharmacy'
os.makedirs(PBPK_POLY_DIR, exist_ok=True)

poly_file = os.path.join(PBPK_POLY_DIR, 'pbpk_polypharmacy_inputs.csv')
poly_df.to_csv(poly_file, index=False)
print(f'PBPK polypharmacy input table saved: {poly_file}')
display(poly_df.head())

# ----------------------------------
# 6. Optional: separate files for age groups
# ----------------------------------
for group in poly_df['age_group_pbpk'].unique():
    group_df = poly_df[poly_df['age_group_pbpk'] == group]
    if not group_df.empty:
        file_path = os.path.join(PBPK_POLY_DIR, f'pbpk_polypharmacy_{group}.csv')
        group_df.to_csv(file_path, index=False)
        print(f'Saved PBPK polypharmacy input for age group {group}: {file_path}')


Harmonized dataset preview:


,study_id,drug,subject_id,age_years,age_days,weight_kg,cl_l_per_h_per_kg,t_half_h,route,aggregated_row,...,cl_ml_per_min_per_kg,cl_l_per_h,cmin_ng_per_ml,visit,dose_mg,auc_mg_h_per_l,cmax_mg_per_l,route_harmonized,pathway,pathway_harmonized
0,vet_et_al_2014_agg,midazolam,NaN,0.006,2.2,2.5,0.140,NaN,iv,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,intravenous,CYP3A4 hepatic metabolism,hepatic_CYP3A4
1,vet_et_al_2014_agg,midazolam,NaN,2.000,730.0,12.0,0.280,NaN,iv,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,intravenous,CYP3A4 hepatic metabolism,hepatic_CYP3A4
2,morph_iv_surgery_1998_agg,morphine,morph_1998_1_1,0.005,NaN,NaN,0.552,NaN,iv,True,...,9.2,NaN,NaN,NaN,NaN,NaN,NaN,intravenous,UGT2B7 hepatic metabolism,hepatic_UGT2B7
3,morph_iv_surgery_1998_agg,morphine,morph_1998_1_2,0.005,NaN,NaN,0.552,NaN,iv,True,...,9.2,NaN,NaN,NaN,NaN,NaN,NaN,intravenous,UGT2B7 hepatic metabolism,hepatic_UGT2B7
4,morph_iv_surgery_1998_agg,morphine,morph_1998_1_3,0.005,NaN,NaN,0.552,NaN,iv,True,...,9.2,NaN,NaN,NaN,NaN,NaN,NaN,intravenous,UGT2B7 hepatic metabolism,hepatic_UGT2B7


PBPK polypharmacy input table saved: /Users/cmontefusco/Pediatric PK Data Repository/data/curated/pbpk_inputs/polypharmacy/pbpk_polypharmacy_inputs.csv


,subject_id,age_group_pbpk,drug,cl_l_per_h_per_kg,weight_kg,polypharmacy
0,g1,preterm_neonate,[gentamicin],0.036,3.3,False
1,meto_1,child,[metoprolol],NaN,NaN,False
2,meto_2,adolescent,[metoprolol],NaN,NaN,False
3,meto_3,child,[metoprolol],NaN,NaN,False
4,morph_1998_1_1,preterm_neonate,[morphine],0.552,NaN,False


Saved PBPK polypharmacy input for age group preterm_neonate: /Users/cmontefusco/Pediatric PK Data Repository/data/curated/pbpk_inputs/polypharmacy/pbpk_polypharmacy_preterm_neonate.csv
Saved PBPK polypharmacy input for age group child: /Users/cmontefusco/Pediatric PK Data Repository/data/curated/pbpk_inputs/polypharmacy/pbpk_polypharmacy_child.csv
Saved PBPK polypharmacy input for age group adolescent: /Users/cmontefusco/Pediatric PK Data Repository/data/curated/pbpk_inputs/polypharmacy/pbpk_polypharmacy_adolescent.csv
Saved PBPK polypharmacy input for age group term_neonate: /Users/cmontefusco/Pediatric PK Data Repository/data/curated/pbpk_inputs/polypharmacy/pbpk_polypharmacy_term_neonate.csv
Saved PBPK polypharmacy input for age group infant: /Users/cmontefusco/Pediatric PK Data Repository/data/curated/pbpk_inputs/polypharmacy/pbpk_polypharmacy_infant.csv
